# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head(20)


,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
5,83106,Hope,Bowers,6484922,229,Bread - Hot Dog Buns,22
6,11253,Nicolas,Holmes,4878478,229,Bread - Hot Dog Buns,3
7,35107,Judith,Miranda,2353899,229,Bread - Hot Dog Buns,9
8,15088,Ruby,Lowe,37021,229,Bread - Hot Dog Buns,4
9,26031,Josephine,Curry,2327232,229,Bread - Hot Dog Buns,7


In [40]:
list(data.columns)

['CustomerID',
 'FirstName',
 'LastName',
 'SalesID',
 'ProductID',
 'ProductName',
 'Quantity']

In [47]:
data['CustomerID']

0        61288
1        77352
2        40094
3        23548
4        78981
         ...  
68579    94547
68580    57654
68581    36751
68582    43270
68583    78718
Name: CustomerID, Length: 68584, dtype: int64

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
quantitydf=data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})

quantitydf.head(110)

#data.groupby('month', as_index=False).agg({"duration": "sum"})
#data.groupby(['month', 'network_type'])['date'].count()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
200        Quiche Assorted                       1
           Rambutan                              1
           Remy Red                              2
           Salmon - Atlantic, Skin On            1
           Sauce - Gravy, Au Jus, Mix            1

[110 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
quantitydf.pivot_table(index="ProductName",values="Quantity",columns="CustomerID",fill_value=0)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [37]:
pivot=quantitydf.pivot_table(index="ProductName",values="Quantity",columns="CustomerID",fill_value=0)
pivot

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Yoghurt Tubes,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,25,0,0
"Yogurt - Blueberry, 175 Gr",0,1,0,0,0,0,0,0,0,0,...,25,0,0,25,0,0,0,0,0,0


In [7]:
from scipy.spatial.distance import pdist, squareform

In [8]:
distances = pdist(pivot.T, 'euclidean')

In [9]:

print(squareform(distances))

[[  0.          11.91637529  10.48808848 ... 228.62851966 239.
  229.77380181]
 [ 11.91637529   0.          11.74734012 ... 228.01096465 239.03765394
  229.70415756]
 [ 10.48808848  11.74734012   0.         ... 228.08112592 238.26665734
  229.77380181]
 ...
 [228.62851966 228.01096465 228.08112592 ...   0.         304.13812651
  305.16389039]
 [239.         239.03765394 238.26665734 ... 304.13812651   0.
  303.10889132]
 [229.77380181 229.70415756 229.77380181 ... 305.16389039 303.10889132
    0.        ]]


In [10]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'euclidean'))), 
                         index=pivot.columns, columns=pivot.columns)

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [11]:
similarities = distances[33].sort_values(ascending=False)[1:6]
similarities

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [12]:
new=quantitydf.loc[[264,3535,3317,2503,3305]]


In [13]:
new.loc[[264]]

Quantity
CustomerID ProductName                            
264        Apricots - Halves                     1
           Apricots Fresh                        1
           Bacardi Breezer - Tropical            1
           Bagel - Plain                         1
           Banana - Leaves                       1
...                                            ...
           Wine - Chardonnay South               1
           Wine - Gato Negro Cabernet            1
           Wine - Redchard Merritt               1
           Wine - Sogrape Mateus Rose            1
           Wine - Wyndham Estate Bin 777         1

[62 rows x 1 columns]

In [22]:
 #new.loc[[3535]]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [15]:
rankedproducts=new.sort_values(by='Quantity', ascending=False)
#df.sort_values(by='col1', ascending=False)

In [16]:
rankedproducts

Quantity
CustomerID ProductName                                
2503       Butter - Unsalted                         2
           Bar - Granola Trail Mix Fruit Nut         2
3305       Wine - Ej Gallo Sierra Valley             2
2503       Knife Plastic - White                     2
264        Guinea Fowl                               2
...                                                ...
2503       Soup - Campbells, Cream Of                1
           Soup - Canadian Pea, Dry Mix              1
           Tea - Decaf Lipton                        1
           Tia Maria                                 1
3535       Yoghurt Tubes                             1

[270 rows x 1 columns]

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [68]:
mergeranked= rankedproducts.merge(pivot[33], on='ProductName').groupby(by='ProductName').sum()
mergeranked

,Quantity,33
ProductName,,
Anchovy Paste - 56 G Tube,1,0
Appetizer - Mushroom Tart,1,0
Appetizer - Sausage Rolls,1,0
Apricots - Dried,1,1
Apricots - Halves,1,0
...,...,...
"Wine - White, Schroder And Schyl",1,0
Wine - Wyndham Estate Bin 777,2,0
Wonton Wrappers,1,0


In [74]:
mergeranked[mergeranked[33]==0].sort_values(by='Quantity', ascending=False)

,Quantity,33
ProductName,,
Wine - Blue Nun Qualitatswein,3,0
Soup - Campbells Bean Medley,3,0
Wine - Ej Gallo Sierra Valley,3,0
Butter - Unsalted,3,0
Yoghurt Tubes,2,0
...,...,...
Fuji Apples,1,0
Garlic - Elephant,1,0
Halibut - Steaks,1,0


In [77]:
top5=mergeranked[mergeranked[33]==0].sort_values(by='Quantity', ascending=False).head(5)
top5

,Quantity,33
ProductName,,
Wine - Blue Nun Qualitatswein,3,0
Soup - Campbells Bean Medley,3,0
Wine - Ej Gallo Sierra Valley,3,0
Butter - Unsalted,3,0
Yoghurt Tubes,2,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [94]:
#data['CustomerID']
dictionary={}
lista=[(customer) for customer in data['CustomerID']]
norepetidos=set(lista)
print(len(norepetidos))

1000


In [97]:
for customer in norepetidos:
    similaritiesall=distances[customer].sort_values(ascending=False)[1:6]
    similarities_item=list(similaritiesall.index)
    newall=quantitydf.loc[similarities_item]
    rankedproductsall=newall.sort_values(by='Quantity', ascending=False)
    mergeranked= rankedproductsall.merge(pivot[customer], on='ProductName').groupby(by='ProductName').sum()
    top5=mergeranked[mergeranked[customer]==0].sort_values(by='Quantity', ascending=False).head(5)
    
    products=list(top5.index)
    dictionary[customer]=products


print(dictionary)


{83973: ['Cocoa Butter', 'Veal - Slab Bacon', 'Bandage - Fexible 1x3', 'Ezy Change Mophandle', 'Pop Shoppe Cream Soda'], 59399: ['Shrimp - Baby, Warm Water', 'Knife Plastic - White', 'Towels - Paper / Kraft', 'Nantucket - Pomegranate Pear', 'Cookie Chocolate Chip With'], 92168: ['Cup - Translucent 7 Oz Clear', 'Snapple Lemon Tea', 'Veal - Brisket, Provimi,bnls', 'Pop Shoppe Cream Soda', 'Spinach - Baby'], 49159: ['Pasta - Penne, Rigate, Dry', 'Cookies - Assorted', 'Cheese - Parmesan Grated', 'Puree - Mocha', 'Rice - Jasmine Sented'], 18441: ['Soupfoamcont12oz 112con', 'Garlic - Primerba, Paste', 'Lemonade - Natural, 591 Ml', 'Sausage - Breakfast', 'Beef - Texas Style Burger'], 22536: ['Cheese - Mozzarella', 'Bacardi Breezer - Tropical', 'Chocolate - Dark', 'Table Cloth - 53x69 Colour', 'Banana - Leaves'], 86028: ['Spinach - Baby', 'Vinegar - Tarragon', 'Pork - Hock And Feet Attached', 'Orange - Canned, Mandarin', 'Bread Foccacia Whole'], 75791: ['Blueberries', 'Vaccum Bag 10x13', 'Turk

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [100]:
pd.DataFrame(dictionary).T

,0,1,2,3,4
83973,Cocoa Butter,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Pop Shoppe Cream Soda
59399,"Shrimp - Baby, Warm Water",Knife Plastic - White,Towels - Paper / Kraft,Nantucket - Pomegranate Pear,Cookie Chocolate Chip With
92168,Cup - Translucent 7 Oz Clear,Snapple Lemon Tea,"Veal - Brisket, Provimi,bnls",Pop Shoppe Cream Soda,Spinach - Baby
49159,"Pasta - Penne, Rigate, Dry",Cookies - Assorted,Cheese - Parmesan Grated,Puree - Mocha,Rice - Jasmine Sented
18441,Soupfoamcont12oz 112con,"Garlic - Primerba, Paste","Lemonade - Natural, 591 Ml",Sausage - Breakfast,Beef - Texas Style Burger
...,...,...,...,...,...
77814,Cheese - Victor Et Berthold,Sea Bass - Whole,Tilapia - Fillets,Cake - Cake Sheet Macaroon,Wine - Ruffino Chianti
32753,Grouper - Fresh,Sherry - Dry,"Chestnuts - Whole,canned",Napkin White - Starched,Yoghurt Tubes
20476,Soup - Campbells Tomato Ravioli,"Pasta - Detalini, White, Fresh",Wanton Wrap,"Wine - Magnotta, Merlot Sr Vqa",Lime Cordial - Roses
67582,Cake - Mini Cheesecake,Ecolab - Solid Fusion,Mussels - Frozen,Wine - Redchard Merritt,Thermometer Digital


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [106]:
distancecityblock = pd.DataFrame(1/(1 + squareform(pdist(pivot.T, 'cityblock'))), 
                         index=pivot.columns, columns=pivot.columns)

#distancecityblock

In [104]:

dictionarycityblock={}

for customer in norepetidos:
    similaritiesall=distancecityblock[customer].sort_values(ascending=False)[1:6]
    similarities_item=list(similaritiesall.index)
    newall=quantitydf.loc[similarities_item]
    rankedproductsall=newall.sort_values(by='Quantity', ascending=False)
    mergeranked= rankedproductsall.merge(pivot[customer], on='ProductName').groupby(by='ProductName').sum()
    top5=mergeranked[mergeranked[customer]==0].sort_values(by='Quantity', ascending=False).head(5)
    
    products=list(top5.index)
    dictionarycityblock[customer]=products


print(dictionarycityblock)

{83973: ['Pork - Bacon, Double Smoked', 'Spinach - Baby', 'Knife Plastic - White', 'Olive - Spread Tapenade', 'Bar - Granola Trail Mix Fruit Nut'], 59399: ['Pepper - Black, Whole', 'Soupfoamcont12oz 112con', 'Pork - Bacon, Double Smoked', 'Butter - Unsalted', 'Salsify, Organic'], 92168: ['Knife Plastic - White', 'Chinese Foods - Chicken', 'Soup - Campbells Bean Medley', 'Wine - Toasted Head', 'Wine - Magnotta, Merlot Sr Vqa'], 49159: ['Veal - Sweetbread', 'Tia Maria', 'Wine - Blue Nun Qualitatswein', 'Butter - Unsalted', 'Table Cloth 81x81 White'], 18441: ['Anchovy Paste - 56 G Tube', 'Bar - Granola Trail Mix Fruit Nut', 'Foam Cup 6 Oz', 'Flour - Pastry', 'Lemonade - Natural, 591 Ml'], 22536: ['Knife Plastic - White', 'Cheese - Mozzarella', 'Sausage - Breakfast', 'Olive - Spread Tapenade', 'Butter - Unsalted'], 86028: ['Knife Plastic - White', 'Sole - Dover, Whole, Fresh', 'Sauce - Hollandaise', 'Foam Dinner Plate', 'Beef - Montreal Smoked Brisket'], 75791: ['Butter - Unsalted', 'Chees

In [107]:
print(similarities_item)

[3317, 2556, 639, 3253, 2503]


In [105]:
pd.DataFrame(dictionarycityblock).T

,0,1,2,3,4
83973,"Pork - Bacon, Double Smoked",Spinach - Baby,Knife Plastic - White,Olive - Spread Tapenade,Bar - Granola Trail Mix Fruit Nut
59399,"Pepper - Black, Whole",Soupfoamcont12oz 112con,"Pork - Bacon, Double Smoked",Butter - Unsalted,"Salsify, Organic"
92168,Knife Plastic - White,Chinese Foods - Chicken,Soup - Campbells Bean Medley,Wine - Toasted Head,"Wine - Magnotta, Merlot Sr Vqa"
49159,Veal - Sweetbread,Tia Maria,Wine - Blue Nun Qualitatswein,Butter - Unsalted,Table Cloth 81x81 White
18441,Anchovy Paste - 56 G Tube,Bar - Granola Trail Mix Fruit Nut,Foam Cup 6 Oz,Flour - Pastry,"Lemonade - Natural, 591 Ml"
...,...,...,...,...,...
77814,Squid - Tubes / Tenticles 10/20,Pork - Kidney,Wanton Wrap,Bandage - Fexible 1x3,Lettuce - Spring Mix
32753,Butter - Unsalted,Veal - Sweetbread,Hersey Shakes,Pork - Inside,Tray - 16in Rnd Blk
20476,"Pasta - Detalini, White, Fresh","Pepper - Black, Whole",Knife Plastic - White,"Cheese - Boursin, Garlic / Herbs",Muffin - Carrot Individual Wrap
67582,Pork - Kidney,Lettuce - Spring Mix,Pomello,"Pasta - Detalini, White, Fresh","Bar Mix - Pina Colada, 355 Ml"
